In [ ]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional
import os
import requests
import json

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

In [ ]:

class REPLResponse(BaseModel):
    answer: str = Field(..., description="Result of the Python execution")
    steps: List[str] = Field(default_factory=list, description="Steps executed by the REPL")

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-8b-8192",
    temperature=0.5
)

def python_repl_tool(expression: str) -> str:
    try:
        repl = PythonREPLTool()
        result = repl.run(expression)
        if not result.strip():
            result = str(eval(expression, {"__builtins__": {}}, {}))
        return json.dumps({
            "answer": result,
            "steps": [f"Executed Python code: {expression}"]
        })
    except Exception as e:
        return json.dumps({
            "answer": f"Error: {str(e)}",
            "steps": [f"Attempted to execute: {expression}"]
        })

tools = [
    Tool(
        name="Python_REPL",
        func=python_repl_tool,
        description="Executes Python expressions like '2 + 3' or 'len([1, 2, 3])'. Returns JSON with 'answer' and 'steps'."
    )
]

custom_prompt = PromptTemplate(
    input_variables=["input"],
    template="Execute the Python expression '{input}' using the Python_REPL tool. Return a JSON object with 'answer' (the result) and 'steps' (a list with the executed code). Complete the task in one step and do not parse the JSON further. Ensure the output is valid JSON."
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"prompt": custom_prompt}
)

def process_query(user_input):
    try:
        if not user_input.strip():
            return "Please enter a valid query."
        response = agent.run(user_input)
        try:
            result = json.loads(response)
            repl_response = REPLResponse(**result)
            output = f"**Answer**: {repl_response.answer}\n\n**Steps**:\n" + "\n".join([f"- {step}" for step in repl_response.steps])
            return output
        except json.JSONDecodeError:
            return f"Error: Invalid response format from agent: {response}"
        except ValueError as ve:
            return f"Error: Invalid JSON structure: {str(ve)}"
    except Exception as e:
        return f"Error processing query: {str(e)}"

with gr.Blocks(title="Python REPL Agent") as demo:
    gr.Markdown("# Simple Python Calculator Agent")
    gr.Markdown("Enter a Python expression like '2 + 3' or 'len([1, 2, 3])' to execute.")
    user_input = gr.Textbox(
        label="Your Query",
        placeholder="e.g., 2 + 3 or len([1, 2, 3])"
    )
    submit_button = gr.Button("Execute")
    output = gr.Textbox(label="Result", lines=5)
    submit_button.click(
        fn=process_query,
        inputs=user_input,
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
Thought: I need to execute a simple arithmetic expression. Python_REPL should be able to handle this.

Action: Python_REPL
Action Input: 2 + 2
Observation: {"answer": "4", "steps": ["Executed Python code: 2 + 2"]}
Thought:Thought: Now that I've executed the expression, I need to extract the answer from the JSON result.

Action: Python_REPL
Action Input: result["answer"]
Observation: {"answer": "NameError(\"name 'result' is not defined\")", "steps": ["Executed Python code: result[\"answer\"]"]}
Thought:Thought: I need to define the variable "result" before trying to access its value.

Action: Python_REPL
Action Input: result = {"answer": "4", "steps": ["Executed Python code: 2 + 2"]}
Observation: {"answer": "Error: invalid syntax (<string>, line 1)", "steps": ["Attempted to execute: result = {\"answer\": \"4\", \"steps\": [\"Executed Python code: 2 + 2\"]}"]}
Thought:Action: Python_REPL
Action Input: result = json.loads(result = {"answer": "4", "s

In [ ]:
# Live currency & stock analyser with web search + python code
# building a research assistant using langchain with multi-tool intelligent agent